# Define a 1-layer convolutional neural network

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class GOLCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Conv2d(1, 1, 3)
        kernel = torch.tensor([[2, 2, 2],
                               [2, 1, 2],
                               [2, 2, 2]])
        kernel = kernel.float().unsqueeze(0).unsqueeze(0)
        self.conv.weight = torch.nn.Parameter(kernel)

    def activation(self, x):
        return torch.heaviside(x - 4.5, torch.tensor(1.0)) - torch.heaviside(x - 7.5, torch.tensor(1.0))

    def forward(self, x):
        x = F.pad(x, (1, 1, 1, 1), mode='circular')
        return self.activation(self.conv(x))

golcnn = GOLCNN()

# Random initial state

In [10]:
import numpy as np
width, height = 100, 100
state = np.random.binomial(n=1, p=0.37, size=(width, height)) # https://arxiv.org/abs/1407.1006
state = state.astype('float32')
state = torch.tensor(state)
state = state.unsqueeze(0).unsqueeze(0)

# (Optional) Push to GPU

In [11]:
device = torch.device('cuda')
golcnn = golcnn.to(device)
state = state.to(device)

# Simulation

In [12]:
import time
history = [state]
t0 = time.time()
for _ in range(1000):
    state = golcnn(state)
    history.append(state)
t1 = time.time()
print(f'Elapsed time: {t1 - t0} s')

Elapsed time: 0.41646599769592285 s


# Save results to video


In [13]:
import cv2
history = torch.stack(history)
history = history.squeeze(1).squeeze(1)
history = history.detach().cpu().numpy()
out = cv2.VideoWriter('output.mp4', fourcc=cv2.VideoWriter_fourcc(*'mp4v'),
                      fps=30, frameSize=(width, height), isColor=False)
for a in history:
    out.write((255*a).astype(np.uint8))
out.release()